# Data Wrangling

## Data Import

In [248]:
import pandas as pd
import numpy as np

In [249]:
#import covariate datasets qnd the outcomes. 

main_table_path = '/Users/alecstear/Documents/GitHub/anti-diabetics-study-with-EHR/data tables/main table analysis/'

cancer_path = f'{main_table_path}cancer_diab_pop.csv'
df_cancer = pd.read_csv(cancer_path)

sex_dates_path = f'{main_table_path}sex_dates.csv'
df_sex_dates = pd.read_csv(sex_dates_path)

COPD_path = f'{main_table_path}COPD_diab_pop.csv'
df_COPD = pd.read_csv(COPD_path)

diabetes_cohort_path = f'{main_table_path}diab_pop.csv'
df_diabetes_cohort = pd.read_csv(diabetes_cohort_path)

exposure_drug_path = f'{main_table_path}exposure_drugs_ATC.csv'
df_exp_drug = pd.read_csv(exposure_drug_path)
#get the table for the exposure. 

fracture_path = f'{main_table_path}fracture_diab_pop.csv'
df_fracture = pd.read_csv(fracture_path)

hypertension_path = f'{main_table_path}hypertension_diab_pop.csv'
df_hypertension = pd.read_csv(hypertension_path)

surgery_path = f'{main_table_path}surgery_diab_pop.csv'
df_surgery = pd.read_csv(surgery_path)

outcome_VTE_path = f'{main_table_path}outcome_VTE.csv'
df_outcome = pd.read_csv(outcome_VTE_path)

#add event date table : outcome_VTE

## All Cohort individuals table

In [250]:
#Calculate the age of our patients. 
df_sex_dates["diab_date"] = pd.to_datetime(df_sex_dates["diab_date"])
df_sex_dates["pat_birth_date"] = pd.to_datetime(df_sex_dates["pat_birth_date"])
df_sex_dates['years of age'] = (df_sex_dates["diab_date"] - df_sex_dates["pat_birth_date"]).dt.days/ 365.25

In [251]:
df_diabetes_cohort = df_sex_dates

In [252]:
#What does this do ? 
#df_diabetes_cohort = df_diabetes_cohort.rename(columns={'pat_id': 'PAT_ID'})
#df_outcome = df_outcome.rename(columns={'pat_id': 'PAT_ID'})
#total_cohort_df = pd.merge(df_diabetes_cohort, df_age_sex, on='PAT_ID', how='inner')
#df_diabetes_cohort = total_cohort_df

## Covariates

In [253]:
#Add the different covariates. 
df_diabetes_cohort['cancer'] = df_diabetes_cohort['PAT_ID'].isin(df_cancer['PAT_ID']).astype(int)
df_diabetes_cohort['COPD'] = df_diabetes_cohort['PAT_ID'].isin(df_COPD['PAT_ID']).astype(int)
df_diabetes_cohort['fracture'] = df_diabetes_cohort['PAT_ID'].isin(df_fracture['PAT_ID']).astype(int)
df_diabetes_cohort['hypertension'] = df_diabetes_cohort['PAT_ID'].isin(df_hypertension['PAT_ID']).astype(int)
df_diabetes_cohort['surgery'] = df_diabetes_cohort['PAT_ID'].isin(df_surgery['PAT_ID']).astype(int)
print(df_diabetes_cohort['cancer'].sum(), df_diabetes_cohort['COPD'].sum(), df_diabetes_cohort['fracture'].sum(), df_diabetes_cohort['hypertension'].sum(), df_diabetes_cohort['surgery'].sum())

1129 416 88 1779 679


## Exposure 

In [254]:
#Add this df_exp_drug using our prescription date and drug ATC Code, with an exposure column. Then add collumn just for the name of the drug prescribed. 

In [255]:
#Using ATC Codes replace ATC Code by the drug class 
dict_ATC_code = {}
dict_ATC_code[" A10BA02"]='Biguanides'
dict_ATC_code[' A10BB01']="Sulfonylureas"
dict_ATC_code[' A10BB07']="Sulfonylureas"
dict_ATC_code[' A10BB12']="Sulfonylureas"
dict_ATC_code[" A10BB09"]="Sulfonylureas"
dict_ATC_code[" A10BH01"]="Dipeptidyl peptidase 4 (DPP-4) inhibitors"
dict_ATC_code[" A10BH02"]="Dipeptidyl peptidase 4 (DPP-4) inhibitors"
dict_ATC_code[" A10BF01"]="Alpha glucosidase inhibitors"
dict_ATC_code[" A10BF02"]="Alpha glucosidase inhibitors"

#Create the new cdrug class column and drop the old code column
df_exp_drug['drug_class'] = df_exp_drug['DRUG_ATC_C07'].map(dict_ATC_code)
df_exp_drug = df_exp_drug.drop(columns=['DRUG_ATC_C07'])
df_exp_drug = df_exp_drug.rename(columns={'prs_date': 'exposure_prescription_date'})
df_exp_drug

,PAT_ID,exposure_prescription_date,drug_class
0,10000008910752101,2019-06-29,Biguanides
1,10001443446743793,2019-05-03,Biguanides
2,10003741345221536,2020-01-05,Biguanides
3,10009136008935799,2020-03-21,Biguanides
4,10011180510471332,2020-02-12,Sulfonylureas
...,...,...,...
6720,29987427320037846,2019-01-28,Sulfonylureas
6721,29995168593291644,2020-01-24,Biguanides
6722,29995597307026117,2019-09-13,Biguanides
6723,29997018692944873,2019-06-10,Biguanides


In [256]:
df_diabetes_cohort

,PAT_ID,PAT_SEX_COD,diab_date,pat_birth_date,years of age,cancer,COPD,fracture,hypertension,surgery
0,10000008910752101,1,2018-03-06,1958-09-13,59.477070,0,0,0,0,0
1,10001443446743793,1,2018-09-17,1944-04-14,74.425736,0,0,0,0,0
2,10003741345221536,1,2018-05-16,1968-12-01,49.453799,0,0,0,1,0
3,10009136008935799,1,2018-05-07,1955-06-28,62.858316,0,0,0,0,0
4,10011180510471332,1,2018-08-29,1943-02-22,75.515400,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8054,29987427320037846,1,2018-07-22,1945-01-27,73.481177,1,0,0,0,0
8055,29995168593291644,2,2018-05-09,1951-05-03,67.017112,0,1,0,1,0
8056,29995597307026117,1,2018-04-18,1949-03-16,69.089665,1,0,0,1,0
8057,29997018692944873,1,2018-06-24,1936-01-20,82.425736,0,0,0,1,0


In [257]:
counter = 0
for i in df_diabetes_cohort['PAT_ID']:
    # Check if 'pat_id' is in 'PAT_ID' column of df_outcome
    if i in df_exp_drug['PAT_ID'].values:
        counter += 1
        # Update 'prs_date' in df_diabetes_cohort based on the condition
        df_diabetes_cohort.loc[df_diabetes_cohort['PAT_ID'] == i, 'exposure_prescription_date'] = df_exp_drug.loc[df_exp_drug['PAT_ID'] == i, 'exposure_prescription_date'].values[0]
        df_diabetes_cohort.loc[df_diabetes_cohort['PAT_ID'] == i, 'exposure_drug_class'] = df_exp_drug.loc[df_exp_drug['PAT_ID'] == i, 'drug_class'].values[0]
counter

6725

In [258]:
#We then One Hot Encode our different exposures for easier statistical analysis in R 
df_diabetes_cohort['exposure_biguanide'] = (df_diabetes_cohort['exposure_drug_class'] == 'Biguanides').astype(int)
df_diabetes_cohort['exposure_Sulfonylureas'] = (df_diabetes_cohort['exposure_drug_class'] == "Sulfonylureas").astype(int)
df_diabetes_cohort['exposure_DPP-4'] = (df_diabetes_cohort['exposure_drug_class'] == "Dipeptidyl peptidase 4 (DPP-4) inhibitors").astype(int)
df_diabetes_cohort['exposure_Alpha glucosidase inhibitors'] = (df_diabetes_cohort['exposure_drug_class'] == "Alpha glucosidase inhibitors").astype(int)

## OUTCOME

In [259]:
#Create VTE (yes/no) and VTE date columns. 

In [260]:
df_outcome

,pat_id,eventdate
0,10009136008935799,2020-08-01
1,10013876319320574,2019-05-03
2,10028221240002707,2020-05-17
3,10033964981130542,2019-09-13
4,10039013233044176,2019-03-26
...,...,...
1978,29965324262070436,2020-01-24
1979,29968111979491266,2019-08-06
1980,29987427320037846,2019-05-03
1981,29995597307026117,2020-12-12


In [261]:
counter = 0
for i in df_diabetes_cohort['PAT_ID']:
    # Check if 'pat_id' is in 'PAT_ID' column of df_outcome
    if i in df_outcome['pat_id'].values:
        counter += 1
        # Update 'prs_date' in df_diabetes_cohort based on the condition
        df_diabetes_cohort.loc[df_diabetes_cohort['PAT_ID'] == i, 'VTE_event_date'] = df_outcome.loc[df_outcome['pat_id'] == i, 'eventdate'].values[0]
counter

1983

In [262]:
#We add a new One Hot encoding column for if our patients have a VTE outcome. 
df_diabetes_cohort['VTE_outcome'] = np.where(df_diabetes_cohort['VTE_event_date'].isnull(), 0, 1)
df_diabetes_cohort

,PAT_ID,PAT_SEX_COD,diab_date,pat_birth_date,years of age,cancer,COPD,fracture,hypertension,surgery,exposure_prescription_date,exposure_drug_class,exposure_biguanide,exposure_Sulfonylureas,exposure_DPP-4,exposure_Alpha glucosidase inhibitors,VTE_event_date,VTE_outcome
0,10000008910752101,1,2018-03-06,1958-09-13,59.477070,0,0,0,0,0,2019-06-29,Biguanides,1,0,0,0,NaN,0
1,10001443446743793,1,2018-09-17,1944-04-14,74.425736,0,0,0,0,0,2019-05-03,Biguanides,1,0,0,0,NaN,0
2,10003741345221536,1,2018-05-16,1968-12-01,49.453799,0,0,0,1,0,2020-01-05,Biguanides,1,0,0,0,NaN,0
3,10009136008935799,1,2018-05-07,1955-06-28,62.858316,0,0,0,0,0,2020-03-21,Biguanides,1,0,0,0,2020-08-01,1
4,10011180510471332,1,2018-08-29,1943-02-22,75.515400,0,0,0,0,0,2020-02-12,Sulfonylureas,0,1,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8054,29987427320037846,1,2018-07-22,1945-01-27,73.481177,1,0,0,0,0,2019-01-28,Sulfonylureas,0,1,0,0,2019-05-03,1
8055,29995168593291644,2,2018-05-09,1951-05-03,67.017112,0,1,0,1,0,2020-01-24,Biguanides,1,0,0,0,NaN,0
8056,29995597307026117,1,2018-04-18,1949-03-16,69.089665,1,0,0,1,0,2019-09-13,Biguanides,1,0,0,0,2020-12-12,1
8057,29997018692944873,1,2018-06-24,1936-01-20,82.425736,0,0,0,1,0,2019-06-10,Biguanides,1,0,0,0,2019-05-22,1


In [263]:
#export latest csv file
csv_file_path = '/Users/alecstear/Documents/GitHub/anti-diabetics-study-with-EHR/data tables/df_MAIN_cohort.csv'
df_diabetes_cohort.to_csv(csv_file_path, index=False)